In [138]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
import statsmodels.api as sm

In [139]:
#read in CSV
bay = pd.read_csv('/home/rgreen/NDVI_Projection/Somalia_Bay_ts_03_18.csv')

In [140]:
bay

,Date,Precip,LST,NDVI
0,1/1/03,0.0000,40.8820,0.4789
1,1/11/03,0.0000,42.1725,0.4103
2,1/21/03,0.0000,46.1791,0.3518
3,2/1/03,0.0000,47.1799,0.3125
4,2/11/03,0.0967,46.4770,0.2904
...,...,...,...,...
571,11/11/18,17.1074,34.7523,0.5274
572,11/21/18,19.9243,37.5115,0.5447
573,12/1/18,24.5147,35.6298,0.5281
574,12/11/18,14.6608,38.6239,0.4909


# 1 Dekad Forecast

In [141]:
bay1 = bay.copy()

In [142]:
#Create Δ NDVI column 
bay1['dN'] = bay1.NDVI.diff()

In [143]:
#convert Date to datetime
bay1['Date'] =  pd.to_datetime(bay1['Date'], format='%m/%d/%y')

In [144]:
#convert date to string and insert new column
bay1.insert(1, 'date_str', bay1['Date'].apply(lambda x: x.strftime('%m-%d-%y')))

In [145]:
bay1

,Date,date_str,Precip,LST,NDVI,dN
0,2003-01-01,01-01-03,0.0000,40.8820,0.4789,NaN
1,2003-01-11,01-11-03,0.0000,42.1725,0.4103,-0.0686
2,2003-01-21,01-21-03,0.0000,46.1791,0.3518,-0.0585
3,2003-02-01,02-01-03,0.0000,47.1799,0.3125,-0.0393
4,2003-02-11,02-11-03,0.0967,46.4770,0.2904,-0.0221
...,...,...,...,...,...,...
571,2018-11-11,11-11-18,17.1074,34.7523,0.5274,0.0564
572,2018-11-21,11-21-18,19.9243,37.5115,0.5447,0.0173
573,2018-12-01,12-01-18,24.5147,35.6298,0.5281,-0.0166
574,2018-12-11,12-11-18,14.6608,38.6239,0.4909,-0.0372


In [146]:
#create list of unique dates (day-month)
day_month=list(bay1['date_str'].str[:5][:36])

In [147]:
#function to create list of climatological averages 
def clim_avg(df,date_list,date_col, variable):

    ndvi_dek_avg = []
    for i in date_list:
        ndvi_avg = df[df[date_col].str.startswith(i)][variable].mean()
        ndvi_dek_avg.append(ndvi_avg)
    return ndvi_dek_avg

In [148]:
ndvi_clim_avg = clim_avg(bay1,day_month,'date_str','NDVI')
precip_clim_avg = clim_avg(bay1,day_month,'date_str','Precip')
lst_clim_avg = clim_avg(bay1,day_month,'date_str','LST')

In [149]:
#insert climatological average columns by repeating over 16 years of data
bay1['NDVI_clim_avg'] = ndvi_clim_avg*16
bay1['Precip_clim_avg'] = precip_clim_avg*16
bay1['LST_clim_avg'] = lst_clim_avg*16

In [150]:
bay1

,Date,date_str,Precip,LST,NDVI,dN,NDVI_clim_avg,Precip_clim_avg,LST_clim_avg
0,2003-01-01,01-01-03,0.0000,40.8820,0.4789,NaN,0.410481,0.065800,42.675131
1,2003-01-11,01-11-03,0.0000,42.1725,0.4103,-0.0686,0.364756,0.021544,43.332106
2,2003-01-21,01-21-03,0.0000,46.1791,0.3518,-0.0585,0.329631,0.016438,44.931481
3,2003-02-01,02-01-03,0.0000,47.1799,0.3125,-0.0393,0.304400,0.000000,45.953906
4,2003-02-11,02-11-03,0.0967,46.4770,0.2904,-0.0221,0.287388,0.011969,46.233281
...,...,...,...,...,...,...,...,...,...
571,2018-11-11,11-11-18,17.1074,34.7523,0.5274,0.0564,0.584462,25.114044,33.866556
572,2018-11-21,11-21-18,19.9243,37.5115,0.5447,0.0173,0.584525,17.976106,35.142000
573,2018-12-01,12-01-18,24.5147,35.6298,0.5281,-0.0166,0.558850,8.393606,36.834869
574,2018-12-11,12-11-18,14.6608,38.6239,0.4909,-0.0372,0.512919,2.297975,39.409119


In [151]:
#create F1-5 columns
bay1['(Nmax-Nt-0)*ln(Pt-0)']=(max(bay.NDVI)-bay1.NDVI_clim_avg)*np.log(1+bay1.Precip_clim_avg)

bay1['(Nmax-Nt-1)*ln(Pt-1)']=(max(bay.NDVI)-bay1['NDVI'].shift(1))*np.log(1+bay1.Precip.shift(1))
bay1['(Nmax-Nt-2)*ln(Pt-2)']=(max(bay.NDVI)-bay1['NDVI'].shift(2))*np.log(1+bay1.Precip.shift(2))
bay1['(Nmax-Nt-3)*ln(Pt-3)']=(max(bay.NDVI)-bay1['NDVI'].shift(3))*np.log(1+bay1.Precip.shift(3))
bay1['(Nmax-Nt-4)*ln(Pt-4)']=(max(bay.NDVI)-bay1['NDVI'].shift(4))*np.log(1+bay1.Precip.shift(4))

In [152]:
bay1

,Date,date_str,Precip,LST,NDVI,dN,NDVI_clim_avg,Precip_clim_avg,LST_clim_avg,(Nmax-Nt-0)*ln(Pt-0),(Nmax-Nt-1)*ln(Pt-1),(Nmax-Nt-2)*ln(Pt-2),(Nmax-Nt-3)*ln(Pt-3),(Nmax-Nt-4)*ln(Pt-4)
0,2003-01-01,01-01-03,0.0000,40.8820,0.4789,NaN,0.410481,0.065800,42.675131,0.019903,NaN,NaN,NaN,NaN
1,2003-01-11,01-11-03,0.0000,42.1725,0.4103,-0.0686,0.364756,0.021544,43.332106,0.007632,0.000000,NaN,NaN,NaN
2,2003-01-21,01-21-03,0.0000,46.1791,0.3518,-0.0585,0.329631,0.016438,44.931481,0.006410,0.000000,0.000000,NaN,NaN
3,2003-02-01,02-01-03,0.0000,47.1799,0.3125,-0.0393,0.304400,0.000000,45.953906,0.000000,0.000000,0.000000,0.000000,NaN
4,2003-02-11,02-11-03,0.0967,46.4770,0.2904,-0.0221,0.287388,0.011969,46.233281,0.005180,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2018-11-11,11-11-18,17.1074,34.7523,0.5274,0.0564,0.584462,25.114044,33.866556,0.451322,0.454740,1.117171,1.006324,0.760856
572,2018-11-21,11-21-18,19.9243,37.5115,0.5447,0.0173,0.584525,17.976106,35.142000,0.406968,0.565941,0.454740,1.117171,1.006324
573,2018-12-01,12-01-18,24.5147,35.6298,0.5281,-0.0166,0.558850,8.393606,36.834869,0.367253,0.541586,0.565941,0.454740,1.117171
574,2018-12-11,12-11-18,14.6608,38.6239,0.4909,-0.0372,0.512919,2.297975,39.409119,0.250453,0.630683,0.541586,0.565941,0.454740


In [155]:
#fill in climatological average of previous months when Nan present due to shift
bay1['(Nmax-Nt-1)*ln(Pt-1)'].iloc[0]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-1])*np.log(1+bay1['Precip_clim_avg'].iloc[-1])

bay1['(Nmax-Nt-2)*ln(Pt-2)'].iloc[0]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-2])*np.log(1+bay1['Precip_clim_avg'].iloc[-2])
bay1['(Nmax-Nt-2)*ln(Pt-2)'].iloc[1]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-1])*np.log(1+bay1['Precip_clim_avg'].iloc[-1])

bay1['(Nmax-Nt-3)*ln(Pt-3)'].iloc[0]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-3])*np.log(1+bay1['Precip_clim_avg'].iloc[-3])
bay1['(Nmax-Nt-3)*ln(Pt-3)'].iloc[1]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-2])*np.log(1+bay1['Precip_clim_avg'].iloc[-2])
bay1['(Nmax-Nt-3)*ln(Pt-3)'].iloc[2]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-1])*np.log(1+bay1['Precip_clim_avg'].iloc[-1])

bay1['(Nmax-Nt-4)*ln(Pt-4)'].iloc[0]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-4])*np.log(1+bay1['Precip_clim_avg'].iloc[-4])
bay1['(Nmax-Nt-4)*ln(Pt-4)'].iloc[1]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-3])*np.log(1+bay1['Precip_clim_avg'].iloc[-3])
bay1['(Nmax-Nt-4)*ln(Pt-4)'].iloc[2]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-2])*np.log(1+bay1['Precip_clim_avg'].iloc[-2])
bay1['(Nmax-Nt-4)*ln(Pt-4)'].iloc[3]=(max(bay1.NDVI)-bay1['NDVI_clim_avg'].iloc[-1])*np.log(1+bay1['Precip_clim_avg'].iloc[-1])

In [156]:
#create F6-10 columns
bay1['(Nt-0-Nmin)*LST']=(bay1.NDVI_clim_avg-min(bay1.NDVI))*(bay1.LST)

bay1['(Nt-1-Nmin)*LST']=(bay1['NDVI'].shift(1)-min(bay1.NDVI))*bay1.LST.shift(1)
bay1['(Nt-2-Nmin)*LST']=(bay1['NDVI'].shift(2)-min(bay1.NDVI))*bay1.LST.shift(2)
bay1['(Nt-3-Nmin)*LST']=(bay1['NDVI'].shift(3)-min(bay1.NDVI))*bay1.LST.shift(3)
bay1['(Nt-4-Nmin)*LST']=(bay1['NDVI'].shift(4)-min(bay1.NDVI))*bay1.LST.shift(4)

In [157]:
#fill in climatological average of previous months when Nan present due to shift
bay1['(Nt-1-Nmin)*LST'].iloc[0]=(bay1['NDVI_clim_avg'].iloc[-1]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-1]

bay1['(Nt-2-Nmin)*LST'].iloc[0]=(bay1['NDVI_clim_avg'].iloc[-2]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-2]
bay1['(Nt-2-Nmin)*LST'].iloc[1]=(bay1['NDVI_clim_avg'].iloc[-1]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-1]

bay1['(Nt-3-Nmin)*LST'].iloc[0]=(bay1['NDVI_clim_avg'].iloc[-3]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-3]
bay1['(Nt-3-Nmin)*LST'].iloc[1]=(bay1['NDVI_clim_avg'].iloc[-2]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-2]
bay1['(Nt-3-Nmin)*LST'].iloc[2]=(bay1['NDVI_clim_avg'].iloc[-1]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-1]

bay1['(Nt-4-Nmin)*LST'].iloc[0]=(bay1['NDVI_clim_avg'].iloc[-4]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-4]
bay1['(Nt-4-Nmin)*LST'].iloc[1]=(bay1['NDVI_clim_avg'].iloc[-3]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-3]
bay1['(Nt-4-Nmin)*LST'].iloc[2]=(bay1['NDVI_clim_avg'].iloc[-2]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-2]
bay1['(Nt-4-Nmin)*LST'].iloc[3]=(bay1['NDVI_clim_avg'].iloc[-1]-min(bay1.NDVI))*bay1['LST_clim_avg'].iloc[-1]

In [158]:
bay1

,Date,date_str,Precip,LST,NDVI,dN,NDVI_clim_avg,Precip_clim_avg,LST_clim_avg,(Nmax-Nt-0)*ln(Pt-0),(Nmax-Nt-1)*ln(Pt-1),(Nmax-Nt-2)*ln(Pt-2),(Nmax-Nt-3)*ln(Pt-3),(Nmax-Nt-4)*ln(Pt-4),(Nt-0-Nmin)*LST,(Nt-1-Nmin)*LST,(Nt-2-Nmin)*LST,(Nt-3-Nmin)*LST,(Nt-4-Nmin)*LST
0,2003-01-01,01-01-03,0.0000,40.8820,0.4789,NaN,0.410481,0.065800,42.675131,0.019903,0.263055,0.250453,0.367253,0.406968,8.805216,10.805787,12.524957,13.398684,13.685173
1,2003-01-11,01-11-03,0.0000,42.1725,0.4103,-0.0686,0.364756,0.021544,43.332106,0.007632,0.000000,0.263055,0.250453,0.367253,7.154828,11.602312,10.805787,12.524957,13.398684
2,2003-01-21,01-21-03,0.0000,46.1791,0.3518,-0.0585,0.329631,0.016438,44.931481,0.006410,0.000000,0.000000,0.263055,0.250453,6.212532,9.075522,11.602312,10.805787,12.524957
3,2003-02-01,02-01-03,0.0000,47.1799,0.3125,-0.0393,0.304400,0.000000,45.953906,0.000000,0.000000,0.000000,0.000000,0.263055,5.156763,7.236265,9.075522,11.602312,10.805787
4,2003-02-11,02-11-03,0.0967,46.4770,0.2904,-0.0221,0.287388,0.011969,46.233281,0.005180,0.000000,0.000000,0.000000,0.000000,4.289246,5.538920,7.236265,9.075522,11.602312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2018-11-11,11-11-18,17.1074,34.7523,0.5274,0.0564,0.584462,25.114044,33.866556,0.451322,0.454740,1.117171,1.006324,0.760856,13.531242,10.155162,7.906494,7.132290,6.341084
572,2018-11-21,11-21-18,19.9243,37.5115,0.5447,0.0173,0.584525,17.976106,35.142000,0.406968,0.565941,0.454740,1.117171,1.006324,14.607916,11.548189,10.155162,7.906494,7.132290
573,2018-12-01,12-01-18,24.5147,35.6298,0.5281,-0.0166,0.558850,8.393606,36.834869,0.367253,0.541586,0.565941,0.454740,1.117171,12.960340,13.114020,11.548189,10.155162,7.906494
574,2018-12-11,12-11-18,14.6608,38.6239,0.4909,-0.0372,0.512919,2.297975,39.409119,0.250453,0.630683,0.541586,0.565941,0.454740,12.275400,11.864723,13.114020,11.548189,10.155162


In [159]:
#X is all Fs
X1 = bay1.iloc[:,7:17][1:]

Y1 = bay1.dN[1:]

In [160]:
#run linear regression
model1 = linear_model.LinearRegression()
model1.fit(X1, Y1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [161]:
#make predictions based on Fs and convert to list to add back into dataframe
predictions1 = list(model1.predict(X1))

In [162]:
#insert np.nan as first element because it is not predicted
predictions1.insert(0,np.nan)

In [163]:
bay1['dN_pred'] = predictions1

In [164]:
#add predicted Δ NDVI and NDVI lag 1 to get predicted NDVI
bay1['NDVI_pred'] = bay1['dN_pred'] + bay1['NDVI'].shift(1)

In [165]:
bay1

,Date,date_str,Precip,LST,NDVI,dN,NDVI_clim_avg,Precip_clim_avg,LST_clim_avg,(Nmax-Nt-0)*ln(Pt-0),...,(Nmax-Nt-2)*ln(Pt-2),(Nmax-Nt-3)*ln(Pt-3),(Nmax-Nt-4)*ln(Pt-4),(Nt-0-Nmin)*LST,(Nt-1-Nmin)*LST,(Nt-2-Nmin)*LST,(Nt-3-Nmin)*LST,(Nt-4-Nmin)*LST,dN_pred,NDVI_pred
0,2003-01-01,01-01-03,0.0000,40.8820,0.4789,NaN,0.410481,0.065800,42.675131,0.019903,...,0.250453,0.367253,0.406968,8.805216,10.805787,12.524957,13.398684,13.685173,NaN,NaN
1,2003-01-11,01-11-03,0.0000,42.1725,0.4103,-0.0686,0.364756,0.021544,43.332106,0.007632,...,0.263055,0.250453,0.367253,7.154828,11.602312,10.805787,12.524957,13.398684,-0.012446,0.466454
2,2003-01-21,01-21-03,0.0000,46.1791,0.3518,-0.0585,0.329631,0.016438,44.931481,0.006410,...,0.000000,0.263055,0.250453,6.212532,9.075522,11.602312,10.805787,12.524957,-0.055086,0.355214
3,2003-02-01,02-01-03,0.0000,47.1799,0.3125,-0.0393,0.304400,0.000000,45.953906,0.000000,...,0.000000,0.000000,0.263055,5.156763,7.236265,9.075522,11.602312,10.805787,-0.038961,0.312839
4,2003-02-11,02-11-03,0.0967,46.4770,0.2904,-0.0221,0.287388,0.011969,46.233281,0.005180,...,0.000000,0.000000,0.000000,4.289246,5.538920,7.236265,9.075522,11.602312,-0.027736,0.284764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2018-11-11,11-11-18,17.1074,34.7523,0.5274,0.0564,0.584462,25.114044,33.866556,0.451322,...,1.117171,1.006324,0.760856,13.531242,10.155162,7.906494,7.132290,6.341084,0.038525,0.509525
572,2018-11-21,11-21-18,19.9243,37.5115,0.5447,0.0173,0.584525,17.976106,35.142000,0.406968,...,0.454740,1.117171,1.006324,14.607916,11.548189,10.155162,7.906494,7.132290,0.009071,0.536471
573,2018-12-01,12-01-18,24.5147,35.6298,0.5281,-0.0166,0.558850,8.393606,36.834869,0.367253,...,0.565941,0.454740,1.117171,12.960340,13.114020,11.548189,10.155162,7.906494,-0.010040,0.534660
574,2018-12-11,12-11-18,14.6608,38.6239,0.4909,-0.0372,0.512919,2.297975,39.409119,0.250453,...,0.541586,0.565941,0.454740,12.275400,11.864723,13.114020,11.548189,10.155162,-0.045170,0.482930


# 4 Dekad Forecast

In [118]:
bay4 = bay.copy()

In [119]:
bay4

,Date,Precip,LST,NDVI
0,1/1/03,0.0000,40.8820,0.4789
1,1/11/03,0.0000,42.1725,0.4103
2,1/21/03,0.0000,46.1791,0.3518
3,2/1/03,0.0000,47.1799,0.3125
4,2/11/03,0.0967,46.4770,0.2904
...,...,...,...,...
571,11/11/18,17.1074,34.7523,0.5274
572,11/21/18,19.9243,37.5115,0.5447
573,12/1/18,24.5147,35.6298,0.5281
574,12/11/18,14.6608,38.6239,0.4909


In [124]:
#Create Δ NDVI column 
bay4['dN'] = bay4.NDVI.diff(4)

In [129]:
#convert Date to datetime
bay4['Date'] =  pd.to_datetime(bay4['Date'], format='%m/%d/%y')

In [130]:
#convert date to string and insert new column
bay4.insert(1, 'date_str', bay4['Date'].apply(lambda x: x.strftime('%m-%d-%y')))

In [131]:
bay4

,Date,date_str,Precip,LST,NDVI,dN
0,2003-01-01,01-01-03,0.0000,40.8820,0.4789,NaN
1,2003-01-11,01-11-03,0.0000,42.1725,0.4103,NaN
2,2003-01-21,01-21-03,0.0000,46.1791,0.3518,NaN
3,2003-02-01,02-01-03,0.0000,47.1799,0.3125,NaN
4,2003-02-11,02-11-03,0.0967,46.4770,0.2904,-0.1885
...,...,...,...,...,...,...
571,2018-11-11,11-11-18,17.1074,34.7523,0.5274,0.1779
572,2018-11-21,11-21-18,19.9243,37.5115,0.5447,0.1742
573,2018-12-01,12-01-18,24.5147,35.6298,0.5281,0.1153
574,2018-12-11,12-11-18,14.6608,38.6239,0.4909,0.0199


In [132]:
#insert climatological average columns by repeating over 16 years of data
bay4['NDVI_clim_avg'] = ndvi_clim_avg*16
bay4['Precip_clim_avg'] = precip_clim_avg*16
bay4['LST_clim_avg'] = lst_clim_avg*16

In [133]:
bay4

,Date,date_str,Precip,LST,NDVI,dN,NDVI_clim_avg,Precip_clim_avg,LST_clim_avg
0,2003-01-01,01-01-03,0.0000,40.8820,0.4789,NaN,0.410481,0.065800,42.675131
1,2003-01-11,01-11-03,0.0000,42.1725,0.4103,NaN,0.364756,0.021544,43.332106
2,2003-01-21,01-21-03,0.0000,46.1791,0.3518,NaN,0.329631,0.016438,44.931481
3,2003-02-01,02-01-03,0.0000,47.1799,0.3125,NaN,0.304400,0.000000,45.953906
4,2003-02-11,02-11-03,0.0967,46.4770,0.2904,-0.1885,0.287388,0.011969,46.233281
...,...,...,...,...,...,...,...,...,...
571,2018-11-11,11-11-18,17.1074,34.7523,0.5274,0.1779,0.584462,25.114044,33.866556
572,2018-11-21,11-21-18,19.9243,37.5115,0.5447,0.1742,0.584525,17.976106,35.142000
573,2018-12-01,12-01-18,24.5147,35.6298,0.5281,0.1153,0.558850,8.393606,36.834869
574,2018-12-11,12-11-18,14.6608,38.6239,0.4909,0.0199,0.512919,2.297975,39.409119


In [134]:
#create F1-5 columns
bay4['Nmax-Nt-0']=(max(bay4.NDVI)-bay4.NDVI_clim_avg)*np.log(1+bay4.Precip_clim_avg)
bay4['Nmax-Nt-1']=(max(bay4.NDVI)-bay4.NDVI_clim_avg)*np.log(1+bay4.Precip_clim_avg)

# bay1['Nmax-Nt-1']=(max(bay.NDVI)-bay1['NDVI'].shift(1))*np.log(1+bay1.Precip.shift(1))
# bay1['Nmax-Nt-2']=(max(bay.NDVI)-bay1['NDVI'].shift(2))*np.log(1+bay1.Precip.shift(2))
# bay1['Nmax-Nt-3']=(max(bay.NDVI)-bay1['NDVI'].shift(3))*np.log(1+bay1.Precip.shift(3))
# bay1['Nmax-Nt-4']=(max(bay.NDVI)-bay1['NDVI'].shift(4))*np.log(1+bay1.Precip.shift(4))

In [135]:
bay4

,Date,date_str,Precip,LST,NDVI,dN,NDVI_clim_avg,Precip_clim_avg,LST_clim_avg,Nmax-Nt-0
0,2003-01-01,01-01-03,0.0000,40.8820,0.4789,NaN,0.410481,0.065800,42.675131,0.019903
1,2003-01-11,01-11-03,0.0000,42.1725,0.4103,NaN,0.364756,0.021544,43.332106,0.007632
2,2003-01-21,01-21-03,0.0000,46.1791,0.3518,NaN,0.329631,0.016438,44.931481,0.006410
3,2003-02-01,02-01-03,0.0000,47.1799,0.3125,NaN,0.304400,0.000000,45.953906,0.000000
4,2003-02-11,02-11-03,0.0967,46.4770,0.2904,-0.1885,0.287388,0.011969,46.233281,0.005180
...,...,...,...,...,...,...,...,...,...,...
571,2018-11-11,11-11-18,17.1074,34.7523,0.5274,0.1779,0.584462,25.114044,33.866556,0.451322
572,2018-11-21,11-21-18,19.9243,37.5115,0.5447,0.1742,0.584525,17.976106,35.142000,0.406968
573,2018-12-01,12-01-18,24.5147,35.6298,0.5281,0.1153,0.558850,8.393606,36.834869,0.367253
574,2018-12-11,12-11-18,14.6608,38.6239,0.4909,0.0199,0.512919,2.297975,39.409119,0.250453
